In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#final model on fashion-mnist
#fashion-mnist
import numpy as np
import cupy as cp
import time
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import accuracy_score
from itertools import product

cp.random.seed(41)
sigmoid = lambda z: 1. / (1. + cp.exp(-z))

# ================== Utilities ==================
def small_world_mask(n_input, n_hidden, k, r, p):
    mask = cp.zeros((n_hidden, n_input))
    for i in range(n_hidden):
        center = cp.random.randint(0, n_input)
        neighbors = [(center + offset) % n_input for offset in range(-r, r+1)]
        selected = neighbors if k > len(neighbors) else cp.random.choice(neighbors, size=k, replace=False)
        mask[i, selected] = 1
        for idx in selected:
            if cp.random.rand() < p:
                mask[i, idx] = 0
                new_idx = cp.random.randint(0, n_input)
                mask[i, new_idx] = 1
    return mask

def train_layer(X_prev, n_hid, C, mask):
    d_prev, N = X_prev.shape
    W_raw = cp.random.randn(n_hid, d_prev)
    W_in = W_raw * mask
    b_in = cp.random.randn(n_hid, 1)

    Z = W_in @ X_prev + b_in
    H = sigmoid(Z)

    HHT = H @ H.T
    I = cp.eye(H.shape[0])
    W_out = cp.linalg.solve(I / C + HHT, H @ X_prev.T)
    return H, (W_in, b_in)

# ================== Main Model ==================
def ml_elm_fashion(k, r, p, p_dist):
    struct = (784, 700, 700, 5000, 10)
    Cs = (1e-1, 1e3, 1e8, 1e8)

    (x_tr, y_tr), (x_te, y_te) = fashion_mnist.load_data()
    x_tr = cp.asarray(x_tr.reshape(-1, 28 * 28).T / 255.)
    x_te = cp.asarray(x_te.reshape(-1, 28 * 28).T / 255.)
    Y_train = cp.asarray((np.eye(10)[y_tr].T) * 2 - 1)

    # Layer 0
    H0 = x_tr

    # Hidden Layer 1
    mask1 = small_world_mask(struct[0], struct[1], k, r, p)
    H1, params1 = train_layer(H0, struct[1], Cs[0], mask1)

    # Hidden Layer 2
    mask2 = small_world_mask(struct[1], struct[2], k, r, p)
    H2, params2 = train_layer(H1, struct[2], Cs[1], mask2)

    # Hidden Layer 3 (distant + sparse)
    H_concat = cp.concatenate((H2, H1), axis=0)
    mask3 = small_world_mask(H_concat.shape[0], struct[3], k, r, p)

    h1_start = H2.shape[0]
    mask3[:, h1_start:] *= (cp.random.rand(struct[3], H1.shape[0]) > p_dist)
    H3, params3 = train_layer(H_concat, struct[3], Cs[2], mask3)

    # Output Layer
    HHT = H3 @ H3.T
    I = cp.eye(H3.shape[0])
    W_out = cp.linalg.solve(I / Cs[-1] + HHT, H3 @ Y_train.T)

    def forward(X):
        Z1 = params1[0] @ X + params1[1]
        H1 = sigmoid(Z1)
        Z2 = params2[0] @ H1 + params2[1]
        H2 = sigmoid(Z2)
        H_concat = cp.concatenate((H2, H1), axis=0)
        Z3 = params3[0] @ H_concat + params3[1]
        H3 = sigmoid(Z3)
        return W_out.T @ H3

    tr_pred = cp.asnumpy(forward(x_tr)).argmax(0)
    te_pred = cp.asnumpy(forward(x_te)).argmax(0)
    tr_acc = accuracy_score(y_tr, tr_pred)
    te_acc = accuracy_score(y_te, te_pred)
    return tr_acc, te_acc

# ================== Run Grid Search ==================
k_list = [8]
r_list = [4]
p_list = [0.05]
p_dist_list = [0.05]

param_grid = list(product(k_list, r_list, p_list, p_dist_list))

print(f"Total Runs: {len(param_grid)}")
best_acc = 0
best_params = None
start_total = time.time()

for i, (k, r, p, p_dist) in enumerate(param_grid):
    print(f"\nTrial {i+1}/{len(param_grid)}: k={k}, r={r}, p={p:.2f}, p_dist={p_dist:.2f}")
    tr_acc, te_acc = ml_elm_fashion(k, r, p, p_dist)
    print(f"Train Acc: {tr_acc*100:.2f}%, Test Acc: {te_acc*100:.2f}%")
    if te_acc > best_acc:
        best_acc = te_acc
        best_params = (k, r, p, p_dist)
    print(f"Best So Far: {best_acc*100:.2f}% with k={best_params[0]}, r={best_params[1]}, p={best_params[2]}, p_dist={best_params[3]}")

print("\n==== Grid Search Completed ====")
print(f"Best Parameters: k={best_params[0]}, r={best_params[1]}, p={best_params[2]}, p_dist={best_params[3]}")
print(f"Best Test Accuracy: {best_acc*100:.2f}%")
print(f"Total Time: {(time.time() - start_total) / 60:.2f} min")

2025-06-26 21:40:42.660484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750974042.836079      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750974042.884359      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Total Runs: 1

Trial 1/1: k=8, r=4, p=0.05, p_dist=0.05
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train Acc: 92.19%, Test Acc: 88.70%
Best So Far: 88.70% with k=8, r=4, p=0.05, p_dist=0.05

==== Grid Search Completed ====
Best Parameters: k=8, r=4, p=0.05, p_dist=0.05
Best Test Accuracy: 88.70%
Total Time: 1.00 min
